In [1]:
import pyspark
print(pyspark.__version__)  # Should print 3.5.5


3.3.2


In [2]:
import py4j
print(py4j.__version__)  # Should print 0.10.9.7


0.10.9.5


In [3]:
from pyspark.sql import SparkSession

# Create a new Spark session (use it directly in case there’s a session issue)
spark = SparkSession.builder \
    .appName("Spark SQL Test") \
    .getOrCreate()

# Test with a simple SQL query
spark.sql("SELECT 1 as test_column").show()


25/03/03 22:00:05 WARN Utils: Your hostname, DESKTOP-RKINDGJ resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/03/03 22:00:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/03/03 22:00:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
+-----------+
|test_column|
+-----------+
|          1|
+-----------+



## Question 3: Count records

In [ ]:
# Load the Parquet file into a DataFrame
df_parquet = spark.read.parquet("yellow_tripdata_2024-10.parquet")

# Print the schema to see the columns
df_parquet.printSchema()


In [6]:
# Load the Parquet file (yellow_tripdata_2024-10.parquet)
df_parquet = spark.read.parquet("yellow_tripdata_2024-10.parquet")

# Load the CSV file (taxi_zone_lookup.csv)
df_csv = spark.read.option("header", "true").csv("taxi_zone_lookup.csv")

# Register DataFrames as SQL temporary views
df_parquet.createOrReplaceTempView("yellow_taxi_trips")
df_csv.createOrReplaceTempView("taxi_zone_lookup")

# Run the SQL query to count records for October 15th
result = spark.sql("""
    SELECT COUNT(*) AS trip_count
    FROM yellow_taxi_trips
    WHERE DATE(tpep_pickup_datetime) = '2024-10-15'
""")

# Show the result
result.show()


+----------+
|trip_count|
+----------+
|    110732|
+----------+



## Question 4: Longest trip

In [7]:
# SQL query to find the longest trip duration in hours
result = spark.sql("""
    SELECT MAX((UNIX_TIMESTAMP(tpep_dropoff_datetime) - UNIX_TIMESTAMP(tpep_pickup_datetime)) / 3600) AS longest_trip_hours
    FROM yellow_taxi_trips
""")

# Show the result
result.show()


+------------------+
|longest_trip_hours|
+------------------+
|162.61777777777777|
+------------------+



## Question 6: Least frequent pickup location zone

In [9]:
result = spark.sql("""
    SELECT tz.Zone, COUNT(*) AS pickup_count
    FROM yellow_taxi_trips yt
    JOIN taxi_zone_lookup tz
    ON yt.PULocationID = tz.LocationID
    GROUP BY tz.Zone
    ORDER BY pickup_count ASC
    LIMIT 1
""")

# Show the result
# result.show()
result.show(truncate=False)

+---------------------------------------------+------------+
|Zone                                         |pickup_count|
+---------------------------------------------+------------+
|Governor's Island/Ellis Island/Liberty Island|1           |
+---------------------------------------------+------------+

